In [1]:
import sys
from PIL import Image as Img
import json
from transformers import AutoModel
import numpy as np
import torch

In [2]:
file_path = 'LoveHina_vol14'

In [3]:
f = open(f'{file_path}.json')

annotation_data = json.load(f)

In [4]:
magi = AutoModel.from_pretrained("ragavsachdeva/magi", trust_remote_code=True).cuda()


2024-04-24 12:34:22.801323: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 12:34:27.205802: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 12:34:27.228853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 12:34:40.251912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
def sort_text_boxes(frames, lines):
    panel_boxes = [(int(panel['xmin']), int(panel['ymin']), int(panel['xmax']), int(panel['ymax'])) for panel in frames]
    text_boxes = [(int(line['attrib']['xmin']), int(line['attrib']['ymin']), int(line['attrib']['xmax']), int(line['attrib']['ymax'])) for line in lines]

    sorted_text_indices = magi.sort_panels_and_text_bboxes_in_reading_order([panel_boxes], [text_boxes])[1][0]
    lines_sorted = [lines[i] for i in sorted_text_indices]

    return lines_sorted


In [6]:
for i, page in enumerate(annotation_data):
    width = int(page['attrib']['width'])

    frames_left = []
    frames_right = []

    lines_left = []
    lines_right = []

    for frame in page['frames']:
        if int(frame['xmin']) < width/2:
            frames_left.append(frame)
        else:
            frames_right.append(frame)

    for line in page['lines']:
        if int(line['attrib']['xmin']) < width/2:
            lines_left.append(line)
        else:
            lines_right.append(line)

    lines_left = sort_text_boxes(frames_left, lines_left)

    lines_right = sort_text_boxes(frames_right, lines_right)

    annotation_data[i]['lines'] = lines_right + lines_left


    

In [7]:
with open(f'{file_path}_fixed_order.json', 'w', encoding='utf-8') as f:
    json.dump(annotation_data, f, ensure_ascii=False, indent=2)